### Retrive table description

Parse the dbt schema file and retrive table description along with column details

In [1]:
import yaml
import pandas as pd

table_description_paths = [
    "dbt/models/dosare/schema/stg_dosare_parti.yml",
    "dbt/models/dosare/schema/stg_dosare.yml",
]

models = []
for path in table_description_paths:
    with open(path, "r") as file:
        data = yaml.safe_load(file)
        for model in data["models"]:
            models.append(
                {
                    "name": model["name"],
                    "description": model["description"],
                    "model": model,
                }
            )

models_df = pd.DataFrame(models)
models_df.head()

,name,description,model
0,stg_dosare_parti,Informații despre părțiile implicate din dosar...,"{'name': 'stg_dosare_parti', 'description': 'I..."
1,stg_dosare,Informații despre dosarele instanțelor judecăt...,"{'name': 'stg_dosare', 'description': 'Informa..."


### Create and store table descripton embeddings

In [2]:
import os
from openai import AzureOpenAI

embeddings_client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version=os.getenv("AZURE_OPENAI_EMBEDDINGS_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

embeddings_model = os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT")


def create_embeddings(text, model=embeddings_model):
    # Create embeddings for each document chunk
    embeddings = (
        embeddings_client.embeddings.create(input=text, model=model).data[0].embedding
    )
    return embeddings


models_df["description_embedding"] = models_df["description"].apply(create_embeddings)
models_df.head()

,name,description,model,description_embedding
0,stg_dosare_parti,Informații despre părțiile implicate din dosar...,"{'name': 'stg_dosare_parti', 'description': 'I...","[0.011188670992851257, 0.017634455114603043, 0..."
1,stg_dosare,Informații despre dosarele instanțelor judecăt...,"{'name': 'stg_dosare', 'description': 'Informa...","[0.001355247339233756, 0.008857620880007744, 0..."


### Generate the sql statement based on user prompt

Call a function that the table descriptions that will return the most likely tables to have this information based a neighboard search.

Feed back the table information to the prompt

Enginier the prompt in such a way that only SQL will be returned using a mult shoot strategy with examples

In [17]:
import pprint
import numpy as np
import json
from sklearn.neighbors import NearestNeighbors

functions = [
    {
        "name": "fetch_tables",
        "description": """Retrieve a list of table information based on the description provided by the user.
          The information will be returned in a json like this:
          [{
            "name": "the name of the table",
            "description": "The description of the info contained in the table",
            "columns": [
              {
                "name": "column name",
                "description: "description of the data in the column",
                "properties": "properties of the column value, e.g. not_null, unique"
              }
            ]
          }]
          """,
        "parametes": {
            "type": "type",
            "properties": {
                "description": {
                    "type": "string",
                    "description": "The description of the information contained in the table in the language of the users prompt or the aproximate name",
                }
            },
            "required": ["description"],
        },
    }
]


def fetch_tables_nearest_neighboards(description):
    embeddings = models_df["description_embedding"].to_list()
    nbrs = NearestNeighbors(n_neighbors=2, algorithm="ball_tree").fit(embeddings)
    description_embedding = create_embeddings(description)
    _, indices = nbrs.kneighbors([description_embedding])

    models = []
    for index in indices[0]:
        models.append(models_df["model"].iloc[index])

    return json.dumps(models, ensure_ascii=False)


def fetch_hardcoded(_description):
    return json.dumps([models_df["model"].iloc[1]], ensure_ascii=False)


def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


def fetch_tables_cosine_similarity(description):
    embeddings = models_df["description_embedding"].to_list()
    models = models_df["model"].to_list()
    description_embedding = create_embeddings(description)

    similarities = [cosine_similarity(description_embedding, emb) for emb in embeddings]
    sorted_models = [
        model
        for _, model in sorted(
            zip(similarities, models), reverse=True, key=lambda x: x[0]
        )
    ]

    return json.dumps(sorted_models, ensure_ascii=False)


result = fetch_hardcoded("dosarele")
print(pprint.pformat(result))

result = fetch_tables_cosine_similarity("dosarele")
print(pprint.pformat(result))

result = fetch_tables_nearest_neighboards("dosarele")
print(pprint.pformat(result))

('[{"name": "stg_dosare", "description": "Informații despre dosarele '
 'instanțelor judecătorești", "columns": [{"name": "obiect", "description": '
 '"Obiectul judecății, acuzațiile", "tests": ["not_null"]}, {"name": '
 '"datainitiala", "description": "Data de înregistrare a dosarului"}, {"name": '
 '"data", "description": "Data ultimei modificări"}, {"name": "parti", '
 '"description": "Numele părții din dosar"}, {"name": "numar", "description": '
 '"Numărul dosarului"}]}]')
('[{"name": "stg_dosare_parti", "description": "Informații despre părțiile '
 'implicate din dosarele instanțelor judecătorești", "columns": [{"name": '
 '"data", "description": "Data de înregistrare a dosarului", "tests": '
 '["not_null"]}, {"name": "nume", "description": "Numele părții din dosar", '
 '"tests": ["not_null"]}]}, {"name": "stg_dosare", "description": "Informații '
 'despre dosarele instanțelor judecătorești", "columns": [{"name": "obiect", '
 '"description": "Obiectul judecății, acuzațiile", "test

In [ ]:
client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

deployment = os.environ["AZURE_OPENAI_DEPLOYMENT"]

available_functions = {
    "fetch_tables": fetch_tables_nearest_neighboards,
}


def generate_sql_for(prompt):
    system = """You are a SQL query expert and you only reply with sql statements.
    If you can't figure out how to create a sql statement from the use prompt just return SELECT 1.
    The queries you generate are always about reading data.
    The queries you generate are never about updating or deleting data.
    Is very important you reply with sql only because your output will be used in a sql interpretern downstream.

    The user will speak to you in their native language
    The user will always ask to retrieve data.
    The user will usually ask you to filter data by something
    The user will ask you sometimes for specific columns from the table

    You have a debug mode, when the user types 'DEBUG: True' at the end of the prompt. I debug mode you provide an explanation of the sql query you generated.

    You have to return the SQL formated with new lines please.

    ## Example 1
    user: Vreau toate dosarele judecatoresti 
    function: [[{"name": "stg_dosare", "description": "Informații despre dosarele '
 'instanțelor judecătorești", "columns": [{"name": "obiect", "description": '
 '"Obiectul judecății, acuzațiile", "tests": ["not_null"]}, {"name": '
 '"datainitiala", "description": "Data de înregistrare a dosarului"}, {"name": '
 '"data", "description": "Data ultimei modificări"}, {"name": "parti", '
 '"description": "Numele părții din dosar"}, {"name": "numar", "description": '
 '"Numărul dosarului"}]}]]
    asssistant: SELECT * FROM stg_dosare

    ## Example 2 
    user: Vreau toate dosarele judecatoresti inregistrate in anul 2023
    function: [[{"name": "stg_dosare", "description": "Informații despre dosarele '
 'instanțelor judecătorești", "columns": [{"name": "obiect", "description": '
 '"Obiectul judecății, acuzațiile", "tests": ["not_null"]}, {"name": '
 '"datainitiala", "description": "Data de înregistrare a dosarului"}, {"name": '
 '"data", "description": "Data ultimei modificări"}, {"name": "parti", '
 '"description": "Numele părții din dosar"}, {"name": "numar", "description": '
 '"Numărul dosarului"}]}]]
    assistant: SELECT * FROM stg_dosare WHERE YEAR(datainitiala) = 2023
    
    ## Example 3 
    user: Vreau toate dosarele judecatoresti cu obiect 'ma injurat de mama'
    function: [[{"name": "stg_dosare", "description": "Informații despre dosarele '
 'instanțelor judecătorești", "columns": [{"name": "obiect", "description": '
 '"Obiectul judecății, acuzațiile", "tests": ["not_null"]}, {"name": '
 '"datainitiala", "description": "Data de înregistrare a dosarului"}, {"name": '
 '"data", "description": "Data ultimei modificări"}, {"name": "parti", '
 '"description": "Numele părții din dosar"}, {"name": "numar", "description": '
 '"Numărul dosarului"}]}]]
    assistant: SELECT * FROM stg_dosare WHERE obiect LIKE '%ma injurat de mama%'
    """

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt},
    ]

    response = client.chat.completions.create(
        model=deployment,
        messages=messages,
        functions=functions,
        function_call="auto",
        temperature=0,
    )

    response_message = response.choices[0].message

    if response_message.function_call and response_message.function_call.name:
        print("Recommended Function call:")
        print(response_message.function_call.name)
        print("With this arguments:")
        print(response_message.function_call.arguments)
        print()

        # Call the function.
        function_name = response_message.function_call.name

        function_to_call = available_functions[function_name]

        function_args = json.loads(response_message.function_call.arguments)
        function_response = function_to_call(**function_args)

        print("Output of function call:")
        print(function_response)
        print(type(function_response))

        # Add the assistant response and function response to the messages
        messages.append(  # adding assistant response to messages
            {
                "role": response_message.role,
                "function_call": {
                    "name": function_name,
                    "arguments": response_message.function_call.arguments,
                },
                "content": None,
            }
        )
        messages.append(  # adding function response to messages
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )

        response = client.chat.completions.create(
            messages=messages,
            model=deployment,
            function_call="auto",
            functions=functions,
            temperature=0,
        )

        response_message = response.choices[0].message

    return response_message




### Give me the keys to the kindom



In [27]:
response_message = generate_sql_for(
    "Vreau evolutia pe luni calendaristice a numarului de dosare judecatoresti in care obiectul contine 'actiune in raspundere delictuala'"
)
print(response_message.content)

SELECT 
    YEAR(datainitiala) AS Anul, 
    MONTH(datainitiala) AS Luna, 
    COUNT(*) AS NumarDosare
FROM stg_dosare
WHERE obiect LIKE '%actiune in raspundere delictuala%'
GROUP BY YEAR(datainitiala), MONTH(datainitiala)
ORDER BY Anul, Luna
